## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [1]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)

sys.path

['/home/nirojasvar/random_sast/ExperimentationRSAST',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python310.zip',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/lib-dynload',
 '',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast/cd_diagram']

In [2]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib.pyplot as plt

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [3]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [4]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [5]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)
        df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
        df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
        df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
        df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


/tmp/ipykernel_22211/1216653100.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_22211/1216653100.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_22211/1216653100.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_22211/1216653100.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_22211/1216653100.py:19: FutureWarning: The default value of

,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.919203,0.181923,0.013529,0.127419,0.037550,0.002808,SonyAIBORobotSurface2,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,half_instance,Rsast,ACF&PACF
1,1,0.828961,0.168610,0.021266,0.037572,0.106363,0.002882,SonyAIBORobotSurface2,ACF&PACF: n_random_points= half_len nb_inst_pe...,half_len,1,Rsast,ACF&PACF
2,2,0.876180,1.377738,0.011656,0.138769,1.211099,0.015296,SonyAIBORobotSurface2,ACF&PACF: n_random_points= half_len nb_inst_pe...,half_len,half_instance,Rsast,ACF&PACF


In [6]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [7]:
#get ds tested overall
df_result['dataset_name'].unique()

array(['SonyAIBORobotSurface2', 'GunPoint', 'Chinatown',
       'DistalPhalanxOutlineCorrect', 'DistalPhalanxOutlineAgeGroup',
       'SmoothSubspace', 'Coffee', 'ShapeletSim', 'ItalyPowerDemand',
       'SyntheticControl'], dtype=object)

In [8]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [9]:
#filter dataset for cd-diagrams with the ds that has already tested all combinations of params
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

complete_ds=complete_ds.tolist()

complete_ds.remove('DistalPhalanxOutlineCorrect')

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()



array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineAgeGroup', 'GunPoint',
       'ItalyPowerDemand', 'SmoothSubspace', 'SonyAIBORobotSurface2',
       'SyntheticControl'], dtype=object)

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [10]:
filter_param=df_result[df_result.rpoint=='half_len']
filter_param.rpoint.unique()

array(['half_len'], dtype=object)

In [11]:
filter_param=filter_param[filter_param.nb_per_class=='half_instance']
filter_param.nb_per_class.unique()

array(['half_instance'], dtype=object)

In [12]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,score,method
0,Chinatown,0.945578,0.971817,0.958017,0.971623,0.966375,0.974927,0.982507,ROCKET
1,Coffee,0.964286,1.000000,0.978571,0.935714,1.000000,1.000000,1.000000,ROCKET
2,DistalPhalanxOutlineAgeGroup,0.734772,0.724700,0.773621,0.729017,0.729496,0.717506,0.748201,ROCKET
3,GunPoint,0.830222,0.976000,0.845778,0.972444,0.965778,0.974667,1.000000,ROCKET
4,ItalyPowerDemand,0.949660,0.946615,0.952705,0.944866,0.951668,0.944866,0.968902,ROCKET
5,SmoothSubspace,0.870222,0.910667,0.879556,0.896889,0.876000,0.908889,0.980000,ROCKET
6,SonyAIBORobotSurface2,0.751242,0.899685,0.756348,0.866387,0.837356,0.893319,0.916055,ROCKET
7,SyntheticControl,0.920000,0.967333,0.896000,0.963333,0.981333,0.966667,1.000000,ROCKET


In [13]:

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='var')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
len_method_vs_ds

# Summary statistics for a Series (single variable)
researchpy.summary_cont(filter_param.groupby(['len_method'])['accuracy'], conf = 0.95)

,N,Mean,SD,SE,95% Conf.,Interval
len_method,,,,,,
ACF,100,0.8565,0.0922,0.0092,0.8382,0.8748
ACF&PACF,100,0.9128,0.0857,0.0086,0.8958,0.9298
Max ACF,100,0.8686,0.0834,0.0083,0.8521,0.8852
Max PACF,100,0.9021,0.0831,0.0083,0.8857,0.9186
None,100,0.8981,0.0897,0.0090,0.8803,0.9159
PACF,100,0.9105,0.0885,0.0088,0.8929,0.9280


In [14]:
stats = filter_param.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF         0.856469    100  0.092215  0.874543  0.838395
ACF&PACF    0.912789    100  0.085700  0.929587  0.895992
Max ACF     0.868632    100  0.083421  0.884983  0.852282
Max PACF    0.902136    100  0.083073  0.918418  0.885854
None        0.898068    100  0.089746  0.915658  0.880477
PACF        0.910459    100  0.088484  0.927802  0.893116


#### Generate Boxplot Tunning Hyperparameter

In [15]:
df_result.nb_per_class.unique()

array(['half_instance', '1', '10', '30', '50', '100'], dtype=object)

In [16]:
df_result.len_method.unique()

array(['ACF&PACF', 'ACF', 'PACF', 'Max ACF', 'Max PACF', 'None'],
      dtype=object)

In [56]:
#generate hyperparameter tuning boxplots

df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','rpoint'], aggfunc='mean')
df_perf=df_perf.reset_index()

for k, ints in enumerate(df_result.nb_per_class.unique()):

    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name'], columns=['rpoint'],values='accuracy')
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        ax.boxplot(df_to_cd,0, '')

        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Boxplot for Len method:'+len_m+' and N° of Intances:'+ints)
        plt.xticks([1, 2, 3,4,5,6],list(df_to_cd) )
        print(list(df_to_cd))

        # save plot
        plt.savefig('images_boxplot/boxplot_'+len_m+ints+'.png')




['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10', '100', '30', '50', 'half_len']
['1', '10',

#### Generate CD Diagram Tunning Hyperparameter

In [18]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
for k,ints in enumerate(df_result.nb_per_class.unique()):
   for len_m in df_result.len_method.unique():
      df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
      print('len_method'+str(len_m))
      print('nb_per_class'+str(ints))
      draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison, nb_inst_per_class="+str(ints), name_file='images_cd_diagram/cd-diagram_'+len_m+ints+'.png')


len_methodACF&PACF
nb_per_classhalf_instance
len_methodACF
nb_per_classhalf_instance


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

len_methodPACF
nb_per_classhalf_instance
the null hypothesis over the entire classifiers cannot be rejected
len_methodMax ACF
nb_per_classhalf_instance
the null hypothesis over the entire classifiers cannot be rejected


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

len_methodMax PACF
nb_per_classhalf_instance
the null hypothesis over the entire classifiers cannot be rejected
len_methodNone
nb_per_classhalf_instance
the null hypothesis over the entire classifiers cannot be rejected


...

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

len_methodMax ACF
nb_per_class100
the null hypothesis over the entire classifiers cannot be rejected
len_methodMax PACF
nb_per_class100
the null hypothesis over the entire classifiers cannot be rejected


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

len_methodNone
nb_per_class100
the null hypothesis over the entire classifiers cannot be rejected


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [19]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
Max ACF: n_random_points=1 nb_inst_per_class=1,214.7500
ACF: n_random_points=1 nb_inst_per_class=1,213.2500
Max ACF: n_random_points=10 nb_inst_per_class=1,207.3125
ACF: n_random_points=10 nb_inst_per_class=1,204.5000
Max PACF: n_random_points=1 nb_inst_per_class=1,204.1250
...,...
ACF&PACF: n_random_points= half_len nb_inst_per_class=100,54.1250
ACF&PACF: n_random_points=100 nb_inst_per_class=half_instance,54.0000
ACF&PACF: n_random_points=30 nb_inst_per_class=100,53.0000
ACF&PACF: n_random_points=100 nb_inst_per_class=50,50.4375


#### Focus on most accurate lenght method

##### Overall Accuracy

In [20]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= half_len nb_inst_per_class=50'], dtype='object')

In [21]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.791263,0.885582,0.908069,0.895154,0.901808
10,NaN,0.861650,0.908505,0.916423,0.917684,0.918451
30,NaN,0.879780,0.904196,0.914956,0.920776,0.920786
50,NaN,0.880732,0.908333,0.919345,0.918831,0.917962
100,NaN,0.881601,0.904856,0.918460,0.922153,0.918959


In [22]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,100,30,50,half_instance
rpoint,,,,,,
1,0.018353,0.006680,0.006496,0.005984,0.007831,0.005389
10,0.007107,0.007133,0.007681,0.008525,0.008367,0.008020
100,0.007060,0.009669,0.008040,0.008131,0.008082,0.006907
30,0.008922,0.009429,0.007986,0.008695,0.007744,0.006754
50,0.005984,0.009068,0.009095,0.007772,0.008464,0.007255
half_len,0.007349,0.008620,0.007569,0.008514,0.008277,0.007344


##### Overall time complexity

In [23]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,
Chinatown,0.000148,0.001009,0.001189,0.000235,0.002611
Coffee,0.001126,0.082135,2.156343,0.000738,2.240458
DistalPhalanxOutlineAgeGroup,0.000529,0.020612,6.829225,0.015880,6.866520
GunPoint,0.000645,0.027494,1.169863,0.000965,1.199090
ItalyPowerDemand,0.000136,0.001336,0.009811,0.000842,0.012150
SmoothSubspace,0.000103,0.002251,0.010263,0.002864,0.015516
SonyAIBORobotSurface2,0.000321,0.002770,0.030471,0.000283,0.033882
SyntheticControl,0.000596,0.022403,3.277018,0.007352,3.307617


In [24]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.015111,0.056451,0.061058,0.075114,0.102111
10,NaN,0.049502,0.154839,0.407523,0.564583,0.823751
30,NaN,0.048544,0.450354,1.193613,1.661248,2.443583
50,NaN,0.079079,0.704976,1.892084,2.611614,3.848694
100,NaN,0.101130,1.011108,2.568373,3.433753,4.919766


##### Calculate weights time complexity

In [25]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50,half_instance
rpoint,,,,,,
1,0.020644,0.020792,0.022511,0.023626,0.023475,0.030458
10,0.021759,0.020989,0.019447,0.017650,0.022379,0.022519
100,0.020841,0.017924,0.022874,0.019713,0.023209,0.021180
30,0.023778,0.020492,0.022524,0.018971,0.022536,0.025112
50,0.018172,0.019975,0.022934,0.015305,0.023215,0.026394
half_len,0.025822,0.027434,0.020311,0.025918,0.022845,0.022103


##### Finding subsequences time complexity

In [26]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50,half_instance
rpoint,,,,,,
1,0.162715,0.497933,1.041109,0.837010,0.914812,1.569111
10,0.043997,0.405674,0.992672,0.693998,0.847386,0.887332
100,0.052281,0.464582,1.253262,0.958145,1.196144,0.898612
30,0.050819,0.461581,1.114042,0.823055,1.007264,0.876790
50,0.044196,0.485267,1.207313,0.889239,1.027739,0.986909
half_len,0.092316,0.733471,1.434441,1.292325,1.442339,0.965750


##### Transform Dataset time complexity

In [27]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50,half_instance
rpoint,,,,,,
1,0.346049,1.110141,5.035816,2.632361,3.537461,4.360863
10,1.065136,8.833083,48.350240,23.698831,32.949706,36.138123
100,5.875026,60.121621,293.667823,152.983376,204.605143,215.845850
30,2.705348,26.493982,145.328837,70.693309,98.532441,107.781804
50,4.543585,41.738543,229.488478,112.511599,155.503506,172.122037
half_len,4.573416,46.756754,204.545282,110.572247,142.527730,154.544750


##### Classifier time complexity

In [28]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50,half_instance
rpoint,,,,,,
1,0.375805,1.755404,0.025696,0.167911,0.029237,0.249031
10,1.837423,0.028465,0.059958,0.038723,0.053221,0.058772
100,0.117877,0.059439,0.230976,0.134306,0.192364,0.186279
30,0.130812,0.041478,0.143987,0.077672,0.107968,0.139268
50,0.136425,0.051516,0.194202,0.103430,0.135921,0.178300
half_len,0.168606,0.058404,0.207417,0.130484,0.184439,0.152924
